In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('MICRODADOS_ENEM_2023.csv', sep=';', encoding='latin-1')

Estrutura do DataFrame

In [ ]:
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608864,210060283376,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,A,A,A,B
608865,210059535906,2023,15,M,2,3,1,1,17,1,...,A,A,A,A,A,A,B,A,B,B
608866,210058701317,2023,4,M,1,1,1,2,0,2,...,A,A,A,B,A,A,B,A,A,B
608867,210060419932,2023,2,F,1,1,1,2,0,3,...,B,B,B,D,B,A,E,A,C,B


Larga colunas que não são importantes para a análise.

In [ ]:
df.drop(['NU_INSCRICAO', 'NU_ANO', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
         'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'CO_MUNICIPIO_ESC'], # O código do município pode ser descartado pois há também uma coluna com o nome do município
         axis=1, inplace=True)

*   A coluna TP_DEPENDENCIA_ADM_ESC indica se a escola é Privada, Federal, Estadual ou Municipal.
*   A coluna NU_NOTA_REDACAO e NU_NOTA_MT indicam a nota da redação e da prova de matemática do aluno.
*   A coluna TP_ENSINO indica o tipo de instituição que o aluno concluiu o ensino médio (normal ou supletivo).

Estas colunas contém informações bem importantes para a classificação, mas estão com vários valores nulos. Como não dá para preencher estes valores com por exemplo a média da coluna, estas linhas serão desconsideradas.

In [ ]:
df.dropna(subset=['TP_DEPENDENCIA_ADM_ESC', 'NU_NOTA_REDACAO', 'NU_NOTA_MT', 'TP_ENSINO'], inplace=True)

Não restam valores nulos nas demais colunas.

In [ ]:
sum( df.isna().sum() )

0

Guarda a coluna TP_DEPENDENCIA_ADM_ESC para análises futuras

In [ ]:
dependencias

Aplica o One-Hot Encoding nas colunas categóricas.

In [ ]:
df = pd.get_dummies(df, drop_first=True,
                    columns=['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
                             'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
                             'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA',
                             'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_LINGUA',
                             'TP_STATUS_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012',
                             'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'])

Categorização das médias, onde a classe i conterá as notas no intevalo [ i×100, (i+1)×100 )


*   Classe 0: [  0, 100)
*   Classe 1: [100, 200)
*   Classe 2: [200, 300) ...



In [ ]:
# Cálculo das médias
medias = (df['NU_NOTA_CH'] + df['NU_NOTA_LC'] +
          df['NU_NOTA_MT'] + df['NU_NOTA_CN'] +
          df['NU_NOTA_REDACAO']) / 5
# Intervalos das médias
intervalos = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# Classifica a média em classes de 0 a 9
df['MÉDIA_CLASSES'] = pd.cut(medias, bins=intervalos, labels=range(10), right=False).astype(int)

Larga as colunas relacionadas a média, já que estão diretamente relacionadas à média.

In [ ]:
df.drop(['NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_REDACAO',
         'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5'], axis=1, inplace=True)

Nova Estrutura do DataFrame.

In [ ]:
df

,TP_FAIXA_ETARIA_2,TP_FAIXA_ETARIA_3,TP_FAIXA_ETARIA_4,TP_FAIXA_ETARIA_5,TP_FAIXA_ETARIA_6,TP_FAIXA_ETARIA_7,TP_FAIXA_ETARIA_8,TP_FAIXA_ETARIA_9,TP_FAIXA_ETARIA_10,TP_FAIXA_ETARIA_11,...,Q022_C,Q022_D,Q022_E,Q023_B,Q024_B,Q024_C,Q024_D,Q024_E,Q025_B,MÉDIA_CLASSES
3,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,5
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4
11,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,6
16,False,False,True,False,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,True,5
41,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219592,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,True,5
219600,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,True,6
219605,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,True,6
219606,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4


Definindo a coluna alvo.

In [64]:
X = df.drop('MÉDIA_CLASSES', axis=1)
y = df['MÉDIA_CLASSES']

Separação dos dados em teste e treino.

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

Treino do modelo.

In [66]:
model = AdaBoostClassifier(random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        34
           2       0.00      0.00      0.00       209
           3       0.00      0.00      0.00      1859
           4       0.00      0.00      0.00      8990
           5       0.43      0.80      0.56     14926
           6       0.46      0.45      0.46      9853
           7       0.00      0.00      0.00      1962
           8       0.00      0.00      0.00        34

    accuracy                           0.43     37868
   macro avg       0.10      0.14      0.11     37868
weighted avg       0.29      0.43      0.34     37868



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


A precisão do modelo foi 0 na maioria das categorias, exceto nas 5 e 6, as duas classes com maior quantidade de valores, onde obteve um resultado moderado.



---



Analisando apenas levando em consideração a dependencia administrativa da escola (Escola Privada, Federal, Estadual, Municipal).

In [69]:
X = df[['TP_DEPENDENCIA_ADM_ESC_2.0', 'TP_DEPENDENCIA_ADM_ESC_3.0', 'TP_DEPENDENCIA_ADM_ESC_4.0']]
y = df['MÉDIA_CLASSES']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
model = AdaBoostClassifier(random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        60
           3       0.00      0.00      0.00       565
           4       0.00      0.00      0.00      3043
           5       0.44      0.81      0.58      5348
           6       0.47      0.46      0.46      3457
           7       0.00      0.00      0.00       641
           8       0.00      0.00      0.00         9

    accuracy                           0.45     13134
   macro avg       0.10      0.14      0.12     13134
weighted avg       0.31      0.45      0.36     13134



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Realizando a análise com base apenas na dependência administrativa da escola, obtivemos resultados bastante próximos dos anteriores, indicando que este é um fator fundamental para esta predição, sendo um fator que influencia bastante nas médias entre 500 e 700 pontos.